# TFM. Modelo predictivo llamadas Call Center
## MVP

## Importacion de librerias 

In [1]:
import pymssql
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

pd.options.display.max_columns = None

### Importacion de los datos de envios por oficinas

In [2]:
df_shippings = pd.read_csv('data/shippings.csv', low_memory=False)

In [3]:
df_shippings['Franquicia destino'].unique()

array([4800, 4802, 4803, 4806, 4810])

In [4]:
df = df_shippings[ (df_shippings['Franquicia destino']==4800) ]
df

,Unnamed: 0,Fecha envío,Número envío,Id. Fiscal,Nombre Comercial,Código servicio,Nombre Rem,Población Rem,CP Rem,Nombre vía Rem,Nombre,Población,Código postal,Nombre vía,Total bultos,Franquicia origen,Franquicia destino,Total,Estado,Tipo anomalía,Motivo,Importe Total
0,0,01/01/2020,00636G662697,A08112716,ACSA OBRAS E INFRAESTRUCT.SA,Urgente 12,ACSA OBRAS E INFRAESTRUCT.SA,BARCELONA,8041,"GUINARDO,DEL",OT00742- ACSA PALMA,PALMA DE MALLORCA,07011,ARGENTINA,1,636,4800,0,Envío cancelado,NaN,Envío borrado,0
1,1,01/01/2020,00640G581369,A08829848,LAB. DR. ECHEVARNE,Económico,ECHEVARNE - BARCELONA,BARCELONA,8037,PROVENÇA,ECHEVARNE - PALMA DE MALLORCA,PALMA DE MALLORCA,07003,ALEMANYA,2,640,4800,0,Envío cancelado,NaN,Envío borrado,0
2,2,01/01/2020,00642G710827,G58236803,INTERMON (M) 4500033714,Económico,INTERMON (M) 4500033714,BARCELONA,8010,"CORTS CATALANES,DE LES",INTERMON,PALMA DE MALLORCA,07001,ARGENTERIA,1,642,4800,0,Envío cancelado,NaN,Recogida anulada (Cancelar envio),0
3,3,01/01/2020,00679G526036,B66587247,ROCK TEETH,Urgente 10,ROCK TEETH-BALEARES,GRANOLLERS,8403,CAN CABANYES,ADESLAS PALMA 1,PALMA DE MALLORCA,07012,RUBEN DARIO,1,679,4800,0,Envío cancelado,NaN,Envío borrado,0
4,4,01/01/2020,026381557689,B47337969,INFOCITEC,E-commerce,INFOCITEC 0101,MADRID,28029,MONFORTE DE LEMOS,"12936 - ALBA CLINICA DENTAL, C",PALMA DE MALLORCA,07003,"JESUS, 3",1,2638,4800,0,Entrega finalizada,NaN,Entrega finalizada,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164142,164142,02/02/2021,04810A015236,B16590515,PISA PISUELA SHOES,Urgente 19,"PISA PISUELA SHOES,S.L.",PALMA DE MALLORCA,7006,PEREZ GALDOS,JAVIER COPETE GARCIA,PEGUERA,07160,BONA VIDA,1,4810,4800,0,Recogida pendiente,NaN,Mensajero de recogida asignado,0
164143,164143,02/02/2021,04810A015329,B57998262,LINEA TERRITORIO BALEAR S.L.,Urgente 19,HOSPITAL DE MURO,MURO,7440,VELER,LINEA TERRITORIO BALEAR S.L.,PALMA DE MALLORCA,07014,RECOGE EN PLATAFORMA,1,4810,4800,"5,570",Recogida finalizada,NaN,Recogida finalizada,0
164144,164144,02/02/2021,04810A015435,B60487915,BRAUN,Urgente 12,HOSPITAL GENERAL DE MURO,PLAYA DE MURO,7458,VELER,braun-TECNOMEDIC,PALMA DE MALLORCA,07014,ANDREA DORIA,1,4810,4800,"9,970",Recogida finalizada,NaN,Recogida finalizada,0
164145,164145,02/02/2021,04810A015464,B07972862,BME. CASTAÑER S.L.,Urgente 19,BME. CASTAÑER S.L.,PALMA DE MALLORCA,7002,SANT MIQUEL,REGISTRO PROPIEDAD CALVIA NUM 1,PALMA NOVA,07181,CORDOBA,1,4810,4800,0,Recogida pendiente,NaN,Mensajero de recogida asignado,0


In [5]:
df['Fecha envío'] = pd.to_datetime(df['Fecha envío']).astype('str')

<ipython-input-5-e4d84bd63bf2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Fecha envío'] = pd.to_datetime(df['Fecha envío']).astype('str')


In [6]:
#Filtro Fecha
#df = df[ df['Fecha envío'] == '2020-02-18' ]

In [7]:
df['Código servicio'].unique()

array(['Urgente 12', 'Económico', 'Urgente 10', 'E-commerce',
       'Urgente 19', 'Mascotas', 'Marítimo baleares', 'Urgente 14',
       'Urgente 19 expedición', 'Urgente 12 expedición', 'Urgente 8:30',
       'Burofax', 'Retorno albarán', 'Bag 14', 'Urgente 10 expedición',
       'Urgente hoy', 'Documentos 14', 'Bag 19',
       'Urgente 8:30 expedición', 'Mrw economy (ecopaq)',
       'Urgente 14 expedición', 'Express documentos', 'Retorno canje'],
      dtype=object)

In [8]:
train = df[['Fecha envío','Código servicio','Franquicia origen','Código postal']]
train.reset_index(inplace=True)
train = train[['Fecha envío','Código servicio','Franquicia origen','Código postal']]

In [9]:
train['Franquicia origen'] = train['Franquicia origen'].apply(str)
train['Código postal'] = train['Código postal'].apply(str)

In [10]:
encoder = OneHotEncoder(handle_unknown='ignore')
enc = train[['Código servicio']]

In [11]:
encoder.fit(enc.values.reshape(-1,1))
encoder.categories_

[array(['Bag 14', 'Bag 19', 'Burofax', 'Documentos 14', 'E-commerce',
        'Económico', 'Express documentos', 'Marítimo baleares', 'Mascotas',
        'Mrw economy (ecopaq)', 'Retorno albarán', 'Retorno canje',
        'Urgente 10', 'Urgente 10 expedición', 'Urgente 12',
        'Urgente 12 expedición', 'Urgente 14', 'Urgente 14 expedición',
        'Urgente 19', 'Urgente 19 expedición', 'Urgente 8:30',
        'Urgente 8:30 expedición', 'Urgente hoy'], dtype=object)]

In [12]:
encoded = encoder.transform(enc.values.reshape(-1,1))
encoded_df = pd.DataFrame(encoded.todense())
encoded_df.columns = encoder.categories_

In [13]:
train = pd.concat([train,encoded_df], axis=1)

In [14]:
encoder = OneHotEncoder(handle_unknown='ignore')
enc = train[['Franquicia origen']]
encoder.fit(enc.values.reshape(-1,1))
encoded = encoder.transform(enc.values.reshape(-1,1))
encoded_df = pd.DataFrame(encoded.todense())
encoded_df.columns = encoder.categories_

In [15]:
train = pd.concat([train,encoded_df], axis=1)

In [16]:
encoder = OneHotEncoder(handle_unknown='ignore')
enc = train[['Código postal']]
encoder.fit(enc.values.reshape(-1,1))
encoded = encoder.transform(enc.values.reshape(-1,1))
encoded_df = pd.DataFrame(encoded.todense())
encoded_df.columns = encoder.categories_

In [17]:
train = pd.concat([train,encoded_df], axis=1)

In [18]:
train.drop(columns=['Código servicio','Franquicia origen','Código postal'], inplace=True)

In [19]:
train = train.groupby(['Fecha envío']).sum()

### Importacion de los datos de llamadas por oficinas

In [20]:
df_calls = pd.read_csv('data/calls.csv')

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [21]:
df_calls = df_calls[['tCreacion','tInicio','tFinal','idCampanya']]

In [22]:
df_calls['tCreacion'] = pd.to_datetime(df_calls['tCreacion'])
df_calls['tInicio'] = pd.to_datetime(df_calls['tInicio'])
df_calls['tFinal'] = pd.to_datetime(df_calls['tFinal'])
df_calls['Date'] = df_calls['tCreacion'].dt.date
df_calls['Date'] = df_calls['Date'].astype('str') 
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186530 entries, 0 to 186529
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   tCreacion   186530 non-null  datetime64[ns]
 1   tInicio     96941 non-null   datetime64[ns]
 2   tFinal      186530 non-null  datetime64[ns]
 3   idCampanya  186530 non-null  float64       
 4   Date        186530 non-null  object        
dtypes: datetime64[ns](3), float64(1), object(1)
memory usage: 7.1+ MB


In [23]:
#df_calls = df_calls[ (df_calls['Date'] == '2020-02-18') & (df_calls['idCampanya'] == 100000021.0 ) ]
df_calls = df_calls[ df_calls['idCampanya'] == 100000021.0 ]

In [24]:
df_calls = df_calls.groupby('Date').count()
df_calls = df_calls[['tCreacion']]

In [26]:
df_explore = pd.concat([train, df_calls], axis=1)
df_explore.dropna(inplace=True)
df_explore

,"(Bag 14,)","(Bag 19,)","(Burofax,)","(Documentos 14,)","(E-commerce,)","(Económico,)","(Express documentos,)","(Marítimo baleares,)","(Mascotas,)","(Mrw economy (ecopaq),)","(Retorno albarán,)","(Retorno canje,)","(Urgente 10,)","(Urgente 10 expedición,)","(Urgente 12,)","(Urgente 12 expedición,)","(Urgente 14,)","(Urgente 14 expedición,)","(Urgente 19,)","(Urgente 19 expedición,)","(Urgente 8:30,)","(Urgente 8:30 expedición,)","(Urgente hoy,)","(100,)","(1000,)","(1002,)","(1003,)","(1004,)","(1005,)","(1006,)","(1007,)","(1008,)","(1009,)","(101,)","(1010,)","(1013,)","(1017,)","(1019,)","(102,)","(103,)","(104,)","(105,)","(1100,)","(1101,)","(1102,)","(1104,)","(1105,)","(1106,)","(1200,)","(1201,)","(1202,)","(1203,)","(1204,)","(1205,)","(1206,)","(1300,)","(1301,)","(1302,)","(1303,)","(1304,)","(1305,)","(1306,)","(1307,)","(1309,)","(1310,)","(1400,)","(1401,)","(1402,)","(1403,)","(1404,)","(1406,)","(1408,)","(1410,)","(1500,)","(1501,)","(1502,)","(1503,)","(1600,)","(1601,)","(1603,)","(1604,)","(1607,)","(1608,)","(1610,)","(1613,)","(1701,)","(1702,)","(1703,)","(1704,)","(1705,)","(1706,)","(1707,)","(1708,)","(1710,)","(1711,)","(1713,)","(1717,)","(1719,)","(1721,)","(1800,)","(1801,)","(1802,)","(1804,)","(1902,)","(1903,)","(1904,)","(1905,)","(1906,)","(1908,)","(1911,)","(2000,)","(2002,)","(2003,)","(2004,)","(2005,)","(201,)","(202,)","(203,)","(204,)","(206,)","(208,)","(209,)","(210,)","(2100,)","(2101,)","(2102,)","(2103,)","(2104,)","(2105,)","(2106,)","(2107,)","(211,)","(212,)","(214,)","(215,)","(216,)","(217,)","(219,)","(220,)","(2201,)","(2202,)","(2204,)","(2205,)","(2206,)","(2208,)","(221,)","(222,)","(223,)","(224,)","(226,)","(2300,)","(2302,)","(2303,)","(2304,)","(2305,)","(2400,)","(2401,)","(2402,)","(2403,)","(2404,)","(2500,)","(2501,)","(2502,)","(2503,)","(2601,)","(2602,)","(2603,)","(2604,)","(2605,)","(2606,)","(2607,)","(2608,)","(2609,)","(2610,)","(2611,)","(2612,)","(2614,)","(2615,)","(2616,)","(2617,)","(2618,)","(2619,)","(2620,)","(2621,)","(2622,)","(2625,)","(2627,)","(2628,)","(2630,)","(2633,)","(2634,)","(2636,)","(2637,)","(2638,)","(2640,)","(2641,)","(2643,)","(2645,)","(2646,)","(2647,)","(2648,)","(2649,)","(2650,)","(2651,)","(2654,)","(2655,)","(2656,)","(2657,)","(2659,)","(2660,)","(2661,)","(2662,)","(2663,)","(2665,)","(2668,)","(2670,)","(2671,)","(2672,)","(2673,)","(2676,)","(2678,)","(2680,)","(2681,)","(2682,)","(2684,)","(2685,)","(2686,)","(2691,)","(2698,)","(2699,)","(2700,)","(2701,)","(2704,)","(2705,)","(2706,)","(2707,)","(2708,)","(2710,)","(2711,)","(2712,)","(2713,)","(2714,)","(2715,)","(2720,)","(2722,)","(2723,)","(2800,)","(2801,)","(2802,)","(2803,)","(2804,)","(2805,)","(2806,)","(2809,)","(2811,)","(2815,)","(2819,)","(2820,)","(2900,)","(2901,)","(2902,)","(2903,)","(2904,)","(2905,)","(300,)","(3000,)","(3002,)","(3005,)","(3007,)","(3008,)","(301,)","(3010,)","(3011,)","(3014,)","(3016,)","(3017,)","(302,)","(303,)","(305,)","(306,)","(307,)","(308,)","(309,)","(310,)","(3100,)","(311,)","(3201,)","(3202,)","(3203,)","(3206,)","(3207,)","(3209,)","(3210,)","(3212,)","(3300,)","(3302,)","(3303,)","(3304,)","(3305,)","(3306,)","(3307,)","(3308,)","(3309,)","(3310,)","(3400,)","(3402,)","(3403,)","(3500,)","(3501,)","(3502,)","(3503,)","(3504,)","(3505,)","(3508,)","(3510,)","(3511,)","(3600,)","(3601,)","(3603,)","(3604,)","(3605,)","(3700,)","(3800,)","(3801,)","(3802,)","(3803,)","(3805,)","(3806,)","(3807,)","(3809,)","(3810,)","(3811,)","(3812,)","(3813,)","(3815,)","(3817,)","(3818,)","(3821,)","(3822,)","(3823,)","(3824,)","(3828,)","(3830,)","(3831,)","(3900,)","(3901,)","(400,)","(4001,)","(4002,)","(4003,)","(4005,)","(4006,)","(4008,)","(4009,)","(4012,)","(4013,)","(4100,)","(4101,)","(4200,)","(4202,)","(4204,)","(4205,)","(4207,)","(4208,)","(4209,)","(4300,)","(4301,)","(4302,)","(4303,)","(4304,)","(4306,)","(4307,)","(4308,)","(4309,)","(4312,)","(4313,)","(4314,)","(4316,)","(4317,)","(4318,)","(4319,)","(4320,)